<a href="https://colab.research.google.com/github/brroavendano/test-churn/blob/main/Copy_of_Ejercicio_5_NLP_Sentiment_Analysis_OAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicio 5 - Análisis de sentimientos en texto no-estructurado (NLP)

La idea es realizar un simple análisis de sentimiento desde comentarios en medios sociales, por medio de un modelo de clasificación supervisada, que entrega dos posibles valores desde el texto: es un comentario POSITIVO o es NEGATIVO. En este caso no se consideran los neutros. Si algún comentario podría ser definido como neutro por un humano, para este ejemplo fue asociado a positivo o negativo.

## Contexto: Análisis de Texto de Comentarios en Medios Sociales de diversos países (Chile, Arg, otros)

Este conjunto de datos generado en 2016 por R:Solver y compartido parcialmente para este tipo de ejercicios, consiste en dos columnas: el texto del comentario, y una clasificación dentro de dos alternativas, positivo o negativo.

El gran objetivo final a resolver con este ejemplo, es lograr predecir el sentimiento, descrito en la variable de clasificación. 


---


## Instrucciones Generales

Todos los alumnos, ya sea en grupo, o individualmente (si no tienen compañeros) deben contestar las preguntas que se indicarán, en un email que se deberá enviar al terminar el ejercicio, **incluyendo, en cada respuesta, un pantallazo del resultado y un comentario intepretativo de esos resultados**.

El formato del contenido del email es el siguiente, considerando que se pide contestar todo dentro del cuerpo del email o en un PDF adjunto como alternativa (No enviar PPT, ni Word, ni tampoco vínculos):

To: rsandova@ing.puc.cl

Subject: Machine Learning - Ejercicio Sentiment Analysis aaaammdd

Integrantes: Apellido Nombre, Apellido Nombre,   (en orden alfabético)

Pregunta 1

(Tabla con resultados de probar las técnicas de normalización)

Conclusión: la más influyente en el resultado es ....

Pregunta 2.1

Para mejorar los resultados se aplicó ...

Pregunta 2.2
(Tabla con resultados de probar diferentes combinaciones)

La mejor proporción entrenamiento/validación es de ...



## Actividad Preparatoria: Instalar librerías de modelos de clasificación

In [1]:
install.packages('e1071')
install.packages('caret')
install.packages('caTools')
install.packages('tm')
install.packages('SnowballC')
install.packages('nnet')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘proxy’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘listenv’, ‘parallelly’, ‘future’, ‘globals’, ‘future.apply’, ‘progressr’, ‘numDeriv’, ‘SQUAREM’, ‘lava’, ‘prodlim’, ‘iterators’, ‘gower’, ‘ipred’, ‘timeDate’, ‘foreach’, ‘plyr’, ‘ModelMetrics’, ‘reshape2’, ‘recipes’, ‘pROC’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘bitops’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘NLP’, ‘slam’, ‘BH’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



## Actividad Preparatoria: Carga de los datos

La siguiente celda de código carga los datos desde la URL de origen y luego muestra un encabezado con las primeras filas del dataset, para demostrar la diposición y ejemplos de los datos. 


In [2]:
# Se declara la URL de dónde obtener los datos
theUrlMain <- "http://RAlize.RSolver.com/RAlize/data/small_sample2019clean.csv"

# Se declaran los nombres de las columnas
columnas <- c("texto","sentimiento")

# Se cargan datos principales a una estructura (commentsdataset), asignando nombres de atributos a las columnas
commentsdataset <- read.csv(file = theUrlMain, header = FALSE, sep = ";", col.names=columnas, skipNul = TRUE)

dim(commentsdataset)
head(commentsdataset, 10)

[1] 4570    2

,texto,sentimiento
,<chr>,<int>
1,David Foronda (Podemos): A favor de regular los transgénicos hay que tener en cuenta la soberanía alimentar.,-1
2,21% la aprueba 73% la rechaza. Así como va Michelle Bachelet desaparecerá de las encuestas ya no la apoyan ni sus votante,-1
3,#CUIDATUDINEROMV #LANUEVAMAYORÍA O #LAVIEJAPILLERÍA #AFP #INJUSTAS EDUARDO #FREI RICARDO #LAGOS Y MICHELLE #BACHELET,-1
4,Ricardo Lagos el mismo que utilizó la LEY ANTITERRORISTA DE PINOCHET para reprimir aplaude a Almagro #NuevaMayoria,-1
5,@estadonacional INSULZA mi candidato presidencial desde gobierno de otro iluminado Ricardo Lagos. CAPACIDAD INTELIGENCIA PERSONALIDAD.,1
6,Venezuela contra los transgénicos: Ley de Semillas recibe respaldo de 28 países,1
7,Como será recordada Michelle Bachelet Aumento del desempleo bajo en economía nula inversión y su nuera Natalia Compagnon. Flor de recuerdo,-1
8,#Longueira habrá pensado que por haber salvado el gobierno corrupto de Ricardo Lagos sería perdonado.,-1
9,@GobiernodeChile @navarrocl y los transgenicos vendrán etiquetados los pollos las hormonas que les introducen ?,-1


## Ejercicio 1: Entender el efecto de cada técnica de normalización de texto

Las técnicas de procesamiento de texto, cuando tienen que realizar "intrepretaciones" del texto leído, deben buscar la simplificación del texto para poder trabajar sobre un universo de texto más simple y normalizado. Esto puede considerar evitar la diferencia de mayúscula/minúsculas, eliminar tildes, evitar palabras comunes, usar la raíz de múltiples términos, entre otros.

A continuación se aplican algunas de estas diferentes técnicas, mostrando el efecto.

**Pregunta 1 (2,5 puntos)**: ¿cuál de todas estas técnicas es la que logra el mayor efecto positivo en los modelos de análisis de sentimiento de la siguiente sección?

Se les pide eliminar (comentar la línea de código respectivo) cada instrucción de normalización y comparar cómo afecta en el accuracy de los modelos más abajo, finalmente determinando cuál es la que tiene mayor efecto. Aquella con más efecto es la que provoca que - al ser eliminada - el accuracy de los modelos implementados más adelante baje más. Se recomienda incluir una tabla de ejecuciones comparadas, que sustente la respuesta.


In [84]:
library(tm)
library(SnowballC)

# Construye el Corpus: el universo de texto que se usará para entrenar los modelos.
corpus.original <- Corpus(VectorSource(commentsdataset$texto))

# Se selecciona y muestra (sin normalizar) un ejemplo aleatorio dentro del corpus
random_index <- floor(runif(1, min=0, max=length(corpus.original)))
content(corpus.original[[random_index]])

################################
# NORMALIZACIÓN DEL TEXTO
# EJERCICIO: probar eliminando (comentando) una por una estas acciones
# para ver cuál tiene mayor efecto en la calidad del modelo de análisis
################################
corpus <- corpus.original # Se saca una copia ('corpus') de trabajo

# Se pasan todas las palabras a minúsculas
 corpus <- tm_map(corpus, tolower)
# Se eliminan todos los signos de puntuación
 corpus <- tm_map(corpus, removePunctuation)
# Se eliminan las stop words (palabras comunes, irrelevantes)
 corpus <- tm_map(corpus, removeWords, c(stopwords("spanish")))
# Se lleva cada palabra a su raíz (stemming)
 corpus <- tm_map(corpus, stemDocument)

# Se muestra el mismo ejemplo aleatorio, pero en texto normalizado
content(corpus[[random_index]])


[1] "Osvaldo Andrade dice que NO ESTUVO en votación en sala en ley que benefició pensión d su esposa Myriam Olate.FALSO"

Warning message in tm_map.SimpleCorpus(corpus, tolower):
“transformation drops documents”
Warning message in tm_map.SimpleCorpus(corpus, removePunctuation):
“transformation drops documents”
Warning message in tm_map.SimpleCorpus(corpus, removeWords, c(stopwords("spanish"))):
“transformation drops documents”
Warning message in tm_map.SimpleCorpus(corpus, stemDocument):
“transformation drops documents”


[1] "osvaldo andrad dice votación sala ley benefició pensión d esposa myriam olatefalso"

## Actividad Complementaria: Construcción de un Vocabulario

Los clasificadores reciben un X de entrada de una dimensión fija. Por lo tanto los X de este ejemplo de análisis de texto, comentarios de cantidad variable de palabras, no se pueden usar tal cual.

Entonces, la siguiente porción de código transforma los comentarios de texto variable en un vector de ocurrencia de palabras desde un vocabulario, el cual se construye referenciando todas las palabras distintas (ya normalizadas) del Corpus. Esto se traduce en que un comentario que tiene la expresión ".. resultados excelentes ... ", tendría una intersección con el comentario "... excelente como resultó ..." y por ello podrían ser interpretadas en forma equivalente. 

Pero el desafío de esta vectorización en base a un vocabulario es la cantidad de dimensiones. Un vocabulario perfectamente puede tener varios miles de palabras diferentes, entonces la dimensión del vector X es de esos varios miles. Por ello, a continuación se busca reducir la dimensionalidad del problema (el tamaño del vocabulario) al reconocer cuáles son los términos más relevantes. Esto se hace con removeSparseTerm y un umbral alto (0.995 reduce la gran cantidad de términos que aparecen en menos de un 99.5% del texto). Mientras mayor es el número, mayor cantidad de términos ocasionales o esporádicos se eliminan. En otras palabras, mientras mayor es el número, mayor es la exigencia para un término de ser considerado valioso.


In [85]:
#######################################################
# Indexación de términos: creación de un Vocabulario
#######################################################

# Primero una matriz de ocurrencia de términos o palabras (DTM: Document-term matrix).
# Las filas son los comentarios y las columnas son las palabras diferentes encontradas.
termMatrix <- DocumentTermMatrix(corpus)
dim(termMatrix)

# Entonces, se eliminan las palabras menos relevantes (sparse terms: términos escasos)
# lo que resulta en una reducción dimensional (potencialmente grande)
termMatrixLight <- removeSparseTerms(termMatrix, 0.995)
dim(termMatrixLight)  # Se puede ver que se reduce significativamente la cantidad de columnas (palabras)

# Re-formatea como un DataFrame
termMatrixDataframe <- as.data.frame(as.matrix(termMatrixLight))

# Se construye el dataset para entrenamiento, agregando la columna del Sentimiento
corpus.procesado <- cbind(commentsdataset, as.matrix(termMatrixLight))
corpus.procesado$sentimiento <- as.factor(commentsdataset$sentimiento)
corpus.procesado$texto <- NULL

# Se muestra el vocabulario y su tamaño
dim(corpus.procesado)


[1] 4570 9289

[1] 4570  372

[1] 4570  373

## Ejercicio 2: Ejecución modelos de predicción según conjuntos de entrenamiento y validación

En este caso, el dataset de ejemplos etiquetados se divide en dos (Hold-out) para entrenar y validar con conjuntos disjuntos.

**Preguntas**

2.1.   (3 puntos) ¿Qué técnica adicional de mejora del dataset de entrenamiento puede lograr mejores resultados?

2.2.   (0,5 puntos) ¿Cuál proporción entre entrenamiento y validación logra los mejores resultados de Accuracy, Sensitivity, Specificity?

In [86]:
library(caTools)

set.seed(12345)

# Incluir aquí las líneas que mejoran el conjunto de entrenamiento y evaluación 
# Esto podría considerar reemplazar las líneas que vienen abajo, que arman estos dos conjuntos en forma simple
clean.data.positivo <- corpus.procesado[corpus.procesado$sentimiento == 1,]
clean.data.negativo <- corpus.procesado[corpus.procesado$sentimiento == -1,] 
balance_ratio <- 1.0
clean.subdata.positivo <- clean.data.positivo
clean.subdata.negativo <- clean.data.negativo[sample(nrow(clean.data.negativo), balance_ratio*dim(clean.data.positivo)[1]), ]

# Por ej, se indican estas dos líneas que recolectan los ejemplos de clase positiva y negativa.
# Al ver su dimensión, se puede apreciar si están o no balanceados
#clean.data.negativo <- corpus.procesado[corpus.procesado$sentimiento == -1,]  
#clean.data.positivo <- corpus.procesado[corpus.procesado$sentimiento == 1,]
dim(clean.data.negativo)
dim(clean.data.positivo)
dim(clean.subdata.negativo)
dim(clean.subdata.positivo)

clean.subdata <- rbind(clean.subdata.positivo, clean.subdata.negativo)

corpus.procesado.working <- clean.subdata

dim(corpus.procesado.working)

ratio <- sample(nrow(corpus.procesado.working),nrow(corpus.procesado.working)*0.90)
training.set = corpus.procesado.working[ratio,]
testing.set  = corpus.procesado.working[-ratio,]





################################################################
# Versión simple para crear conjuntos de entrenamiento y de test
# ratio <- sample(nrow(corpus.procesado),nrow(corpus.procesado)*0.80)
# training.set = corpus.procesado[ratio,]
# testing.set  = corpus.procesado[-ratio,]

 dim(training.set)
 dim(testing.set)

# head(testing.set, 20)

[1] 3401  373

[1] 1169  373

[1] 1169  373

[1] 1169  373

[1] 2338  373

[1] 2104  373

[1] 234 373

## Implementación de modelos de clasificación de referencia

Habiendo definido y establecido los conjuntos de entrenamiento y de test, a continuación se ejecutan unos pocos modelos de clasificación: Decision Tree, Naive Bayes y más abajo y en una sección separada se encuentra una Red Neuronal (NNET). Cada uno obtiene su resultado, mostrando sus indicadores de desempeño.


In [87]:
library(e1071) 
library(rpart)
library(caret)

# Decision Tree
tree.model <- rpart(sentimiento ~ ., data=training.set, method="class", minbucket=10)
tree.predict <- predict(tree.model, testing.set, type = "class")
print("Resultados Árbol de Decisión")
confusionMatrix(tree.predict, as.factor(testing.set$sentimiento)) 

# Naive Bayes
NB_model <- naiveBayes(sentimiento ~ ., data=training.set)
NB_predict <- predict(NB_model, testing.set)
print("Resultados Naive Bayes")
confusionMatrix(NB_predict, as.factor(testing.set$sentimiento)) 

[1] "Resultados Árbol de Decisión"


Confusion Matrix and Statistics

          Reference
Prediction  -1   1
        -1 110 108
        1    2  14
                                          
               Accuracy : 0.5299          
                 95% CI : (0.4638, 0.5953)
    No Information Rate : 0.5214          
    P-Value [Acc > NIR] : 0.4226          
                                          
                  Kappa : 0.0933          
                                          
 Mcnemar's Test P-Value : <2e-16          
                                          
            Sensitivity : 0.9821          
            Specificity : 0.1148          
         Pos Pred Value : 0.5046          
         Neg Pred Value : 0.8750          
             Prevalence : 0.4786          
         Detection Rate : 0.4701          
   Detection Prevalence : 0.9316          
      Balanced Accuracy : 0.5484          
                                          
       'Positive' Class : -1              
                              

[1] "Resultados Naive Bayes"


Confusion Matrix and Statistics

          Reference
Prediction  -1   1
        -1  48  21
        1   64 101
                                          
               Accuracy : 0.6368          
                 95% CI : (0.5716, 0.6984)
    No Information Rate : 0.5214          
    P-Value [Acc > NIR] : 0.0002393       
                                          
                  Kappa : 0.2605          
                                          
 Mcnemar's Test P-Value : 5.225e-06       
                                          
            Sensitivity : 0.4286          
            Specificity : 0.8279          
         Pos Pred Value : 0.6957          
         Neg Pred Value : 0.6121          
             Prevalence : 0.4786          
         Detection Rate : 0.2051          
   Detection Prevalence : 0.2949          
      Balanced Accuracy : 0.6282          
                                          
       'Positive' Class : -1              
                              

In [88]:
library(nnet) 

# Red Neuronal
NN_model <- nnet(as.factor(sentimiento) ~ ., data=training.set, size=20, maxit=120, MaxNWts=20000)
NN_predict <- predict(NN_model, testing.set, type="class")

# A continuación se muestra el resultado de evaluación 
confTable <- table(NN_predict, testing.set$sentimiento)
confTable

accuracy <- (confTable[1,1] + confTable[2,2]) / dim(testing.set)[1]
print('Accuracy')
accuracy

sensitivity <- confTable[1,1] / (confTable[1,1] + confTable[1,2])
print('Sensitiviy')
sensitivity

specificity <- confTable[2,2] / (confTable[2,1] + confTable[2,2])
print('Specificity')
specificity

# weights:  7481
initial  value 1563.317966 
iter  10 value 958.234080
iter  20 value 547.943372
iter  30 value 387.670744
iter  40 value 358.611348
iter  50 value 352.380151
iter  60 value 350.187120
iter  70 value 349.486915
iter  80 value 349.240788
iter  90 value 349.143990
iter 100 value 349.104733
iter 110 value 349.089217
iter 120 value 349.077974
final  value 349.077974 
stopped after 120 iterations


          
NN_predict -1  1
        -1 75 30
        1  37 92

[1] "Accuracy"


[1] 0.7136752

[1] "Sensitiviy"


[1] 0.7142857

[1] "Specificity"


[1] 0.7131783